<a href="https://colab.research.google.com/github/RoboMaroof/LLM-Applications-Building-Blocks/blob/main/03_Quantization/05_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/william2020/llama-cpp-quantization-llama3-1-8b

https://github.com/AIAnytime/GGUF-Quantization-of-any-LLM/blob/main/Quantize_LLMs_to_GGUF%20(1).ipynb

https://medium.com/the-ai-forum/instruction-fine-tuning-gemma-2b-on-medical-reasoning-and-convert-the-finetuned-model-into-gguf-844191f8d329

https://medium.com/@kevin.lopez.91/simple-tutorial-to-quantize-models-using-llama-cpp-from-safetesnsors-to-gguf-c42acf2c537d

https://github.com/ggerganov/llama.cpp/discussions/2948

https://huggingface.co/spaces/ggml-org/gguf-my-repo

https://huggingface.co/spaces/ggml-org/gguf-my-repo/blob/main/app.py

https://www.youtube.com/watch?v=wxQgGK5K0rE

https://towardsdatascience.com/the-ultimate-handbook-for-llm-quantization-88bb7cb0d9d7#2c92

# Clone llama.cpp Repository

In [1]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 32603, done.
remote: Counting objects: 100% (10037/10037), done.
remote: Compressing objects: 100% (764/764), done.
remote: Total 32603 (delta 9717), reused 9317 (delta 9271), pack-reused 22566 (from 1)
Receiving objects: 100% (32603/32603), 55.84 MiB | 19.89 MiB/s, done.
Resolving deltas: 100% (23562/23562), done.


In [2]:
# !cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements/requirements-convert-hf-to-gguf.txt

Makefile:76: *** LLAMA_CUBLAS is removed. Use GGML_CUDA instead..  Stop.


# Installs and Imports

In [3]:
from huggingface_hub import snapshot_download

# Load Base Model

In [4]:
model_name = "google/gemma-2b-it"

In [6]:
base_model = "./original_model/"

In [7]:
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+'/FP16.gguf'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

# Choose Quantization level

In [5]:
methods = 'q4_k_m'

In [8]:
!mkdir ./quantized_model/

# Convert model to GGUF format

In [9]:
!python llama.cpp/convert_hf_to_gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> F16, shape = {2048, 256000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.bfloat16 --> F16, shape = {16384, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.bfloat16 --> F16, shape = {2048, 16384}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.bfloat16 --> F16, shape = {2048, 16384}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,       torch.bfloat16 --> F16, shape = {2048, 256}
INFO:hf-to-gguf:blk.0.attn_output.weight,  torch.bfloat16 --> F16, shape = {2048, 2

# Build llama.cpp

In [19]:
!mkdir llama.cpp/build && cd llama.cpp/build && cmake .. && cmake --build . --config Release

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5")
-- OpenMP found
-- Using llamafile
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCE

# Quantize the Model

In [21]:
!cd llama.cpp/build/bin && ./llama-quantize /content/quantized_model/FP16.gguf /content/quantized_model/Q4_K_M.gguf q4_K_M

main: build = 3619 (8f824ffe)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/quantized_model/FP16.gguf' to '/content/quantized_model/Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 33 key-value pairs and 164 tensors from /content/quantized_model/FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 2.5B
llama_model_loader: - kv   4:                            general.license str              = gemma
llama_model_loader: - kv   5:                

In [22]:
!pip install llama-cpp-python==0.2.85

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.85-cp310-cp310-linux_x86_64.whl size=2873162 sha256=31d62137ad855bc21bfddfef63c5553b1c000bfebdbb4b00b1521a12c70def7b
  Stored in directory: /root/.cache/pip/wheels/3f/e8/4e/29a754f9175ef52b6481cd75e3af4de38bf6dfa9c2972f75d4
Successfully built llama-cpp-python


# Inference using Quantized Model

In [23]:
from llama_cpp import Llama    # Llama to gemma 2b

In [24]:
model_path = "/content/quantized_model/Q4_K_M.gguf"

In [25]:
llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 33 key-value pairs and 164 tensors from /content/quantized_model/Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 2.5B
llama_model_loader: - kv   4:                            general.license str              = gemma
llama_model_loader: - kv   5:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   6:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   7:                          gemma.blo

In [26]:
generation_kwargs = {
    "max_tokens":300,
    "echo":False,
    "top_k":1
}

prompt = "Which country hosted 2018 fifa world cup?"
res = llm(prompt, **generation_kwargs)
res.get("choices")[0].get("text")


llama_print_timings:        load time =    2087.84 ms
llama_print_timings:      sample time =       8.84 ms /    48 runs   (    0.18 ms per token,  5428.02 tokens per second)
llama_print_timings: prompt eval time =    2087.77 ms /    13 tokens (  160.60 ms per token,     6.23 tokens per second)
llama_print_timings:        eval time =   11404.80 ms /    47 runs   (  242.66 ms per token,     4.12 tokens per second)
llama_print_timings:       total time =   13602.42 ms /    60 tokens


'\n\nA) Russia\n\nB) Brazil\n\nC) Germany\n\nD) Argentina\n\nThe 2018 FIFA World Cup was held in Russia from June 13 to July 15, 2018.'

In [33]:
output = llm("Provide Information about world war 2 in 1000 words.", max_tokens=2048, stop=["\n"], echo=False)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2087.84 ms
llama_print_timings:      sample time =       0.33 ms /     1 runs   (    0.33 ms per token,  3048.78 tokens per second)
llama_print_timings: prompt eval time =    2197.55 ms /    15 tokens (  146.50 ms per token,     6.83 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    2204.11 ms /    16 tokens


In [34]:
print(output['choices'][0]['text'])

model size comparison

inference speed comparison

push model to HF

save locally